We need to change our waveform generation in order to fit the scanning frequency.
Current waveform generator is not set to the scanning frequency

We need to know:
1. Scanner frequency (time per scan)
2. Maximum modulation frequency (1MHz for internal, 25MHz for pockels)

We can take the scanner frequency (7.912kHz) and get the period (126.4us)
Assuming the TTL is a single pulse, we can say each scan is (63.2us)
We take the maximum modualtion frequency (1MHz) which is (1us) meaning we can hit 63px per scan
The AWG can handle 100MS/s so to start we can do 25 samples per pixel, meaning each number is actually 25 of them. 
Then we sample at 25MHz

On a trigger, we start the waveform at the right frequency

In [1]:
scan_f = 7911
mod_f = 1_000_000

def generate_waveform(scan_f, mod_f):
    scan_time = 1/scan_f
    mod_time = 1/mod_f
    
generate_waveform(scan_f, mod_f)

In [13]:
wf = []
for _ in range(60):
    for i in range(50):
        wf.append(0.84)
    for i in range(50):
        wf.append(-0.44)

In [71]:
allon = []
for _ in range(60):
    allon.append(-1.0)
    for i in range(9):
        allon.append(1.0)
    allon.append(-1.0)

In [72]:
# Sampling rate / number of points = pulse width

# At 1MHz:
# Up pulse: 50 points (500ns)
# Down pulse = 50 points (500ns)


# For each binary pixel, we need 100 points
# If pixel is white, the first 50 are 1s
# If the pixel is black, they are all 0s

In [21]:
import nifgen
import time

with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = 100_000_000
    wf_handle = session.create_waveform(wf)
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.trigger_mode = nifgen.TriggerMode.STEPPED
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.configure_arb_waveform(wf_handle, gain=1, offset=0)
    session.initiate()
    time.sleep(5)

In [20]:
with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.FUNC
    session.configure_standard_waveform(waveform = nifgen.Waveform.SQUARE, amplitude=0.9, frequency = 7910.95)
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    with session.initiate():
        time.sleep(15)

In [ ]:
# Lets make a list with a bunch of different waveforms

In [27]:
import nifgen
import time
import numpy as np

# parameters
duration = 120e-6              # total waveform length (60 µs)
sample_rate = 100e6           # 100 MHz AWG clock
num_samples = int(duration * sample_rate)

# build a single-cycle square wave (high half, low half)
wf = np.concatenate([
    np.ones(num_samples // 2),       # high
    np.zeros(num_samples - num_samples // 2)  # low
])

with nifgen.Session("Dev1") as session:
    # set up ARB mode
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = sample_rate

    # upload our 60 µs square
    wf_handle = session.create_waveform(wf)

    # trigger config: one waveform per rising edge on PFI1
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.digital_edge_start_trigger_edge = (
        nifgen.StartTriggerDigitalEdgeEdge.RISING
    )
    session.trigger_mode = nifgen.TriggerMode.STEPPED

    # tie the arb into the output
    session.configure_arb_waveform(wf_handle, gain=0.5, offset=0)

    session.initiate()
    # now each PFI1 rising edge → one 60 µs square pulse
    time.sleep(5)


In [26]:
import nifgen
import time
import numpy as np

# parameters
cycles = 120
cycle_duration = 5e-7       # each square cycle is 20 µs
sample_rate = 100e6          # 100 MHz AWG clock

# number of samples per 20 µs cycle
samples_per_cycle = int(cycle_duration * sample_rate)

# build one 50%-duty square cycle (high half, low half)
one_cycle = np.concatenate([
    np.ones(samples_per_cycle // 2),
    np.zeros(samples_per_cycle - samples_per_cycle // 2)
])

# repeat it 3 times in a row
wf = np.tile(one_cycle, cycles)


with nifgen.Session("Dev1") as session:
    # set up ARB mode
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = sample_rate

    # upload our 3×20 µs burst
    wf_handle = session.create_waveform(wf)

    # trigger config: one burst per rising edge on PFI1
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.digital_edge_start_trigger_edge = (
        nifgen.StartTriggerDigitalEdgeEdge.RISING
    )
    session.trigger_mode = nifgen.TriggerMode.STEPPED

    # tie the arb into the output
    session.configure_arb_waveform(wf_handle, gain=0.5, offset=0)

    session.initiate()
    # now each PFI1 rising edge → three 20 µs square pulses
    time.sleep(15)


In [23]:
import nifgen
import time
import numpy as np

# parameters
num_elements    = 120
cycle_duration  = 20e-6       # 20 µs per waveform element
sample_rate     = 100e6       # 100 MHz AWG clock
samples_per_el  = int(cycle_duration * sample_rate)  # = 2000 samples

# build a 50%-duty template waveform
template = np.concatenate([
    np.ones(samples_per_el // 2),
    np.zeros(samples_per_el - samples_per_el // 2)
])

# create 120 distinct waveforms (here just circularly shifted copies)
waveforms = [np.roll(template, i) for i in range(num_elements)]

with nifgen.Session("Dev1") as session:
    # 1) Put AWG into sequence mode
    session.output_mode     = nifgen.OutputMode.SEQ
    session.arb_sample_rate = sample_rate

    # 2) Upload each waveform to the device memory
    handles = [session.create_waveform(wf) for wf in waveforms]

    # 3) Build an ARB sequence: one pass of each handle
    #    create_arb_sequence(waveform_handles, loop_counts)
    seq_handle = session.create_arb_sequence(handles, [1] * num_elements)
    # :contentReference[oaicite:0]{index=0}

    # 4) Configure that sequence (gain/offset)
    session.configure_arb_sequence(seq_handle, gain=0.5, offset=0.0)
    # :contentReference[oaicite:1]{index=1}

    # 5) Stepped trigger on PFI1 rising-edge
    session.start_trigger_type                = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.digital_edge_start_trigger_edge   = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.trigger_mode                      = nifgen.TriggerMode.STEPPED

    # 6) Start generation
    session.initiate()
    # Now each rising edge on PFI1 will advance to the next 20 µs element in your 120-element sequence.
    time.sleep(15)


In [36]:
import os
import time
import numpy as np
import nifgen

def stream_numpy_waveform(data: np.ndarray,
                          resource: str = "Dev1",
                          sample_rate: float = 100e6,
                          waveform_name: str = "streamwave",
                          chunk_size_samples: int = 1_000_000):
    """
    Stream a large numpy array to the AWG in chunks.

    Args:
        data: 1D float32 numpy array of samples.
        resource: NI-FGEN resource string (e.g. "Dev1").
        sample_rate: ARB sample clock in Hz.
        waveform_name: the named waveform in the AWG memory.
        chunk_size_samples: how many samples to send per iteration.
    """
    total_samples = data.size
    i = 0
    with nifgen.Session(resource) as session:
        # use SCRIPT mode for streaming
        session.output_mode = nifgen.OutputMode.SCRIPT
        session.arb_sample_rate = sample_rate

        offset = 0
        while offset < total_samples:
            # slice out next chunk
            chunk = data[offset : offset + chunk_size_samples]
            offset += chunk.size

            # ensure correct dtype
            chunk = np.asarray(chunk, dtype=np.float32)

            # allocate & write
            waveform_name = f"{waveform_name}{i}"
            session.allocate_named_waveform(waveform_name, chunk.size)
            session.write_waveform(waveform_name, chunk.tolist())
            i += 1
            # write a tiny script to generate it once
            session.write_script(f"""
script main
    repeat 1
        generate {waveform_name}
    end repeat
end script
""")
            # initiate & wait
            with session.initiate():
                session.wait_until_done()


def generate_large_test_waveform(duration_s: float,
                                 sample_rate: float = 100e6,
                                 freq_hz: float = 1e6) -> np.ndarray:
    """
    Generate a large sine-wave test pattern.

    Args:
        duration_s: total length of waveform in seconds.
        sample_rate: desired sample rate in Hz.
        freq_hz: sine frequency in Hz.

    Returns:
        float32 numpy array of length = duration_s * sample_rate
    """
    total_samples = int(duration_s * sample_rate)
    t = np.arange(total_samples) / sample_rate
    # simple sine wave, cast to float32
    return np.sin(2 * np.pi * freq_hz * t, dtype=np.float32)


In [38]:
# 1) generate 0.5 s of 1 MHz sine at 100 MHz
wf = generate_large_test_waveform(10, sample_rate=100e6, freq_hz=1e6)

# 2) stream it in 1 M-sample chunks
stream_numpy_waveform(wf,
                      resource="Dev1",
                      sample_rate=100e6,
                      chunk_size_samples=1_000_000)


DriverError: -1074126844: The function generator's waveform memory is full.

Waveform Name: streamwave0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134
Waveform Length: 2000000
Total Device Memory: 268435456
Device Memory Available: 435328
Largest Available Memory Block: 435328
Memory Used by Waveforms: 268000000
Memory Used by Procedures: 64
Units: Bytes

Status Code: -200315

In [55]:
import numpy as np
import nifgen

def stream_numpy_waveform(data: np.ndarray,
                          resource: str      = "Dev1",
                          sample_rate: float = 100e6,
                          waveform_name: str = "streamwave",
                          chunk_size_samples: int = 1_000_000):
    """
    Stream a large numpy array to the AWG in chunks, re-using one
    named waveform and guaranteeing each write is exactly the
    allocated length.
    """
    total_samples = data.size

    # make sure our array is 1D float64
    data = np.asarray(data, dtype=np.float64).ravel()

    with nifgen.Session(resource) as session:
        session.output_mode     = nifgen.OutputMode.SCRIPT
        session.arb_sample_rate = sample_rate

        # 1) allocate exactly once at the full chunk size
        session.allocate_named_waveform(waveform_name, chunk_size_samples)

        offset = 0
        while offset < total_samples:
            # 2) slice out up to chunk_size_samples
            remaining = total_samples - offset
            take = min(remaining, chunk_size_samples)
            chunk = data[offset : offset + take]
            offset += take

            # 3) if this slice is too short, pad with zeros
            if take < chunk_size_samples:
                pad = np.zeros(chunk_size_samples - take, dtype=np.float64)
                chunk = np.concatenate([chunk, pad])

            # 4) final sanity: truncate any excess (just in case)
            chunk = chunk[:chunk_size_samples]

            # 5) write exactly chunk_size_samples doubles
            session.write_waveform(waveform_name, chunk.tolist())

            # 6) generate it once via SCRIPT
            session.write_script(f"""
script main
    repeat 1
        generate {waveform_name}
    end repeat
end script
""")

            # 7) fire and wait
            with session.initiate():
                session.wait_until_done()



def generate_large_test_waveform(duration_s: float,
                                 sample_rate: float = 100e6,
                                 freq_hz: float = 1e6) -> np.ndarray:
    """
    Generate a large sine-wave test pattern.

    Args:
        duration_s: total length of waveform in seconds.
        sample_rate: desired sample rate in Hz.
        freq_hz: sine frequency in Hz.

    Returns:
        float32 numpy array of length = duration_s * sample_rate
    """
    total_samples = int(duration_s * sample_rate)
    t = np.arange(total_samples) / sample_rate
    return np.sin(2 * np.pi * freq_hz * t).astype(np.float64)


In [56]:
# 1) Make 10 s of a 1 MHz sine at 100 MHz sample rate
wf = generate_large_test_waveform(10, sample_rate=100e6, freq_hz=1e6)

In [57]:
import numpy as np
import math

# Determining allocation size for a NumPy array
def allocation_size(waveform: np.ndarray, max_chunk_size: int = 10 * 1024 * 1024):
    """
    Given a NumPy array, find the largest chunk size (in bytes) that evenly divides
    the total array size and is less than or equal to `max_chunk_size`.

    Args:
        waveform (np.ndarray): Input array.
        max_chunk_size (int): Maximum allowed chunk size in bytes. Default is 10MB.

    Returns:
        int or None: Largest valid chunk size in bytes, or None if none found.
    """
    total_size_bytes = waveform.nbytes
    valid_chunks = set()

    for i in range(1, int(math.isqrt(total_size_bytes)) + 1):
        if total_size_bytes % i == 0:
            complement = total_size_bytes // i
            if i <= max_chunk_size:
                valid_chunks.add(i)
            if complement <= max_chunk_size:
                valid_chunks.add(complement)

    return max(valid_chunks) if valid_chunks else None


In [58]:
wf.dtype

dtype('float64')

In [103]:
chunk_size = 100000
chunk_size

100000

In [114]:
with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.SCRIPT
    session.arb_sample_rate = sample_rate
    handle = session.allocate_named_waveform("wf", chunk_size)
    session.write_waveform(handle, wf[:chunk_size])
    session.write_script("""
    script main
        repeat forever
            generate wf[:chunk_size]
        end repeat
    end script
    """)
    session.initiate()
    offset = chunk_size
    total  = wf.size
    while True:
        # see how many samples you can now write into the buffer
        if session.streaming_space_available_in_waveform >= chunk_size:
            # grab the next chunk (wrap-round at the end)
            chunk = wf[offset : offset + chunk_size]
            if chunk.size < chunk_size:
                # pad the final piece so it exactly fills the buffer
                chunk = np.pad(chunk, (0, chunk_size - chunk.size), "constant")
            session.write_waveform(handle, chunk)
            offset = (offset + chunk_size) % total

TypeError: 'NoneType' object cannot be interpreted as an integer

In [3]:
import numpy as np
import nifgen

# ■ Configuration
resource_name = "Dev1"
sample_rate   = 100e6                  # Samples per second
chunk_size    = 1_000_000              # Samples per buffer chunk
# wf should be your full waveform as float32; we'll cast to float64 below
wf = np.sin(2 * np.pi * 1e6 * np.arange(10_000_000) / sample_rate).astype(np.float32)

with nifgen.Session(resource_name) as session:
    
    # 3) Configure ARB mode & sample rate
    session.output_mode     = nifgen.OutputMode.SCRIPT
    session.arb_sample_rate = sample_rate
    # 1) Reserve onboard memory for streaming
    handle = session.allocate_named_waveform("streamingwf", chunk_size)
    # 2) Identify it as the streaming waveform
    session.streaming_waveform_handle = handle


    # 4) Load the first chunk (cast to float64)
    session.write_waveform(handle, wf[:chunk_size].astype(np.float64))

    # 5) Start generation
    session.initiate()

    # 6) Stream remaining data chunk-by-chunk
    offset = chunk_size
    total  = wf.size
    while offset < total:
        available = session.streaming_space_available_in_waveform
        if available >= chunk_size:
            # Grab next slice (with wrap-around, if desired)
            chunk = wf[offset : offset + chunk_size]
            if chunk.size < chunk_size:
                chunk = np.pad(chunk, (0, chunk_size - chunk.size), "constant")
            session.write_waveform(handle, chunk.astype(np.float64))
            offset += chunk_size

    # (Optionally) wait until generation completes or call session.abort()


TypeError: 'NoneType' object cannot be interpreted as an integer

In [8]:
import numpy as np
import nifgen
import time

# ■ User parameters
RESOURCE      = "Dev1"
SRATE         = 100e6           # 100 MS/s
CHUNK_SAMPLES = 1_000_000       # samples per onboard chunk
REPEAT_COUNT  = 2               # how many times to play the entire wf
WF_TOTAL      = 10_000_000      # total samples in your waveform

# build a dummy waveform (replace with your data)
t   = np.arange(WF_TOTAL) / SRATE
wf  = (0.5 * np.sin(2 * np.pi * 1e6 * t)).astype(np.float32)
wf64 = wf.astype(np.float64)    # write_waveform wants float64 (or int16)

with nifgen.Session(RESOURCE) as session:
    # 1) ARB mode & clock
    session.output_mode      = nifgen.OutputMode.ARB
    session.arb_sample_rate  = SRATE

    # 2) Allocate an anonymous chunk and mark it streaming
    handle = session.allocate_waveform(CHUNK_SAMPLES)
    if handle is None:
        raise RuntimeError("allocate_waveform failed")
    session.streaming_waveform_handle = handle

    # 3) Tell the instrument how to play that buffer
    session.arb_waveform_start_offset  = 0
    session.arb_waveform_repeat_count = REPEAT_COUNT

    # 4) Prime the buffer with the first chunk
    session.write_waveform(handle, wf64[:CHUNK_SAMPLES])

    # 5) Initiate generation
    session.initiate()

    # 6) Refill as space frees up
    offset = CHUNK_SAMPLES
    total  = wf64.size
    while offset < total * REPEAT_COUNT:
        # wait until there’s room
        while session.streaming_space_available_in_waveform < CHUNK_SAMPLES:
            time.sleep(0.001)
        # calculate slice (wrap-around for repeats)
        idx = offset % total
        block = wf64[idx : idx + CHUNK_SAMPLES]
        if block.size < CHUNK_SAMPLES:
            block = np.pad(block, (0, CHUNK_SAMPLES - block.size), "constant")
        session.write_waveform(handle, block)
        offset += CHUNK_SAMPLES

    # 7) Wait for completion (or abort on timeout)
    try:
        session.wait_until_done(10)
        print("Done streaming.")
    except nifgen.Error:
        print("Timeout/abort, stopping generation.")
        session.abort()


AttributeError: 'Session' object has no attribute 'arb_waveform_start_offset'

In [10]:
import numpy as np
import nifgen
import time

# Configuration
resource_name = "Dev1"
channel_name = "0"
sample_rate = 10_000_000
waveform_size = 1_000_000_000  # 1 billion samples = ~4 GB float32
block_count = 500
repeat_count = 1

# Derived
waveform_block_size = waveform_size // block_count
total_blocks_to_stream = block_count * (repeat_count - 1)

# Generate full waveform timebase
#print("Generating full waveform and chunking...")
#t = np.arange(waveform_size) / sample_rate
#freq = 1e5  # 100 kHz
#full_waveform = np.sign(np.sin(2 * np.pi * freq * t)).astype(np.float32)

# Chunk into list of lists
waveform_chunks = [
    full_waveform[i * waveform_block_size : (i + 1) * waveform_block_size].tolist()
    for i in range(block_count)
]



In [11]:
# Start NI-FGEN session
with nifgen.Session(resource_name) as session:
    print("1. Setting output mode to ARB")
    session.output_mode = nifgen.OutputMode.ARB

    print("2. Setting sample rate")
    session.arb_sample_rate = sample_rate

    print("3. Allocating waveform memory")
    handle = session.allocate_waveform(waveform_block_size)

    print("4. Setting up streaming handle")
    session.streaming_waveform_handle = handle

    print("5. Setting trigger mode to SINGLE")
    session.trigger_mode = nifgen.TriggerMode.SINGLE
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING

    print("6. Setting repeat count")
    session.arb_repeat_count = repeat_count

    print("7. Writing first chunk to start")
    session.write_waveform(handle, waveform_chunks[0])

    print("8. Initiating generation")
    session.initiate()

    print("9. Streaming remaining blocks")
    for i, chunk in enumerate(waveform_chunks[1:]):  # skip first block
        session.write_waveform(handle, chunk)
        streamed_samples += waveform_block_size
        print(f"   → Block {i+2}/{block_count} streamed")

    print("10. Waiting for generation to complete")
    try:
        session.wait_until_done(60.0)
        print("✅ Generation complete")
    except nifgen.errors.DriverError as e:
        session.abort()
        print("⚠️ Generation aborted:", e)

1. Setting output mode to ARB
2. Setting sample rate
3. Allocating waveform memory
4. Setting up streaming handle
5. Setting trigger mode to SINGLE
6. Setting repeat count
7. Writing first chunk to start
8. Initiating generation
9. Streaming remaining blocks



KeyboardInterrupt



In [17]:
import nifgen
import time
import numpy as np

# parameters
cycles = 630
cycle_duration = 63e-8      # each square cycle is 20 µs
sample_rate = 100e6          # 100 MHz AWG clock

# number of samples per 20 µs cycle
samples_per_cycle = int(cycle_duration * sample_rate)

# build one 50%-duty square cycle (high half, low half)
one_cycle = np.concatenate([
    np.ones(samples_per_cycle // 2),
    np.zeros(samples_per_cycle - samples_per_cycle // 2)
])

# repeat it 3 times in a row
wf = np.tile(one_cycle, cycles)

with nifgen.Session("Dev1") as session:
    # set up ARB mode
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = sample_rate

    # upload our 3×20 µs burst
    wf_handle = session.create_waveform(wf)

    # trigger config: one burst per rising edge on PFI1
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.digital_edge_start_trigger_edge = (
        nifgen.StartTriggerDigitalEdgeEdge.RISING
    )
    session.trigger_mode = nifgen.TriggerMode.STEPPED

    # tie the arb into the output
    session.configure_arb_waveform(wf_handle, gain=0.5, offset=0)

    session.initiate()
    # now each PFI1 rising edge → three 20 µs square pulses
    time.sleep(5)


DriverError: -1074115985: Requested waveform length is invalid, because the number of samples is not an integer multiple of the waveform length increment.

Requested Value: 79380
Waveform Length Increment: 4

Status Code: -200400